In [1]:
pwd

'C:\\Users\\mc\\Desktop'

In [2]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver                        # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC     # 웹브라우저 동작
from tqdm import tqdm
from tqdm.notebook import tqdm
import re # 지금 코드에 사용하지는 않음...
from time import sleep
import time

In [3]:
# 데이터 수집할 키워드 지정
keyword = "SK텔레콤"
keyword

'SK텔레콤'

# 다음뉴스들 url 수집

In [4]:
# 크롬창 띄우기
driver = webdriver.Chrome("chromedriver.exe")  # 맥 : /Users/aiden/Desktop/chromedriver
driver.get("https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q={}".format(keyword))
time.sleep(3) #문자 안에 변수를 넣고 싶을떼 넣고 싶은 위치에 {}를 쓰고 뒤에 .format(변수)를 넣어준다

In [5]:
# page1에서 다음뉴스 url 수집하기
things = driver.find_elements_by_link_text('다음뉴스')  

url_list1 = []
for thing in things:
    url1 = thing.get_attribute('href')
    url_list1.append(url1)
    

In [6]:
driver.find_element_by_link_text("2").click( )

In [7]:
# page1에서 다음뉴스 url 수집하기
things = driver.find_elements_by_link_text('다음뉴스')  

url_list2 = []
for thing in things:
    url2 = thing.get_attribute('href')
    url_list2.append(url2)
    

In [8]:
url_list = (url_list1+url_list2)
print(len(url_list))
url_list1

24


['http://v.media.daum.net/v/20210430144039263',
 'http://v.media.daum.net/v/20210430093505411',
 'http://v.media.daum.net/v/20210430082628689',
 'http://v.media.daum.net/v/20210429101914284',
 'http://v.media.daum.net/v/20210429144339853',
 'http://v.media.daum.net/v/20210429091129534',
 'http://v.media.daum.net/v/20210429135420404',
 'http://v.media.daum.net/v/20210429140205821',
 'http://v.media.daum.net/v/20210429082052508',
 'http://v.media.daum.net/v/20210429082052507',
 'http://v.media.daum.net/v/20210429103302072',
 'http://v.media.daum.net/v/20210429161116989',
 'http://v.media.daum.net/v/20210428163405690',
 'http://v.media.daum.net/v/20210428101202047',
 'http://v.media.daum.net/v/20210428092547732']

In [9]:
df = pd.DataFrame({"url":url_list})
df.to_csv('daumnews_urls.csv')

# 뉴스 별 댓글 수집

In [10]:
# 저장해둔 url 불러오기
df = pd.read_csv('daumnews_urls.csv')
df['url']

0     http://v.media.daum.net/v/20210430144039263
1     http://v.media.daum.net/v/20210430093505411
2     http://v.media.daum.net/v/20210430082628689
3     http://v.media.daum.net/v/20210429101914284
4     http://v.media.daum.net/v/20210429144339853
5     http://v.media.daum.net/v/20210429091129534
6     http://v.media.daum.net/v/20210429135420404
7     http://v.media.daum.net/v/20210429140205821
8     http://v.media.daum.net/v/20210429082052508
9     http://v.media.daum.net/v/20210429082052507
10    http://v.media.daum.net/v/20210429103302072
11    http://v.media.daum.net/v/20210429161116989
12    http://v.media.daum.net/v/20210428163405690
13    http://v.media.daum.net/v/20210428101202047
14    http://v.media.daum.net/v/20210428092547732
15    http://v.media.daum.net/v/20210428141101265
16    http://v.media.daum.net/v/20210426155252639
17    http://v.media.daum.net/v/20210426122705886
18    http://v.media.daum.net/v/20210426082246320
19    http://v.media.daum.net/v/20210424083005148


In [11]:
len(df['url'])

24

In [14]:
# 뉴스 크롬창 띄우기
driver = webdriver.Chrome("chromedriver.exe")  # 맥 : /Users/aiden/Desktop/chromedriver
driver.get(df['url'][22])

## 기사 제목,날짜,추천수 수집

In [15]:
# 기사 제목
title = driver.find_element_by_css_selector('.tit_view')
title = title.text
print(title)

회사 쪼개는 SK텔레콤, 통신 넘어 비통신 빅테크로 거듭날까


In [16]:
# 기사 날짜
date = driver.find_element_by_css_selector('.num_date')
date = date.text
print(date)

2021. 04. 24. 20:03


## 기사감정 수집(좋아요, 훈훈해요, 슬퍼요, 화나요)

In [17]:
like = driver.find_element_by_css_selector(".emotion_list .selectionbox.type-LIKE.unselected .count")
like = like.text
print(like)

angry = driver.find_element_by_css_selector(".emotion_list .selectionbox.type-ANGRY.unselected .count")
angry = angry.text
print(angry)

sad = driver.find_element_by_css_selector(".emotion_list .selectionbox.type-SAD.unselected .count")
sad = sad.text
print(sad)


0
1
0


## 댓글 수집

In [18]:
# 기사 댓글 갯수 
review_count = driver.find_element_by_css_selector(".num_count").text
review_count = int(review_count.replace(',', ''))
review_count

1

In [19]:
# 더보기 클릭 횟수
moreview_num = review_count
moreview_num

1

In [20]:
# review 수집하기
review_list = []

overlays1 = ".desc_txt"
reviews = driver.find_elements_by_css_selector(overlays1)

for review in tqdm(reviews):    
    review = review.text
    review_list.append(review)

print(len(review_list))
review_list


1


['ICT 투자전문회사\n최종적으로 SK지주랑 인수합병하는거겠지요?']

In [21]:
target_info = {}

target_info['기사명'] = title
target_info['날짜'] = date
target_info['댓글'] = review_list
target_info['좋아요'] = like
target_info['화나요'] = angry
target_info['슬퍼요'] = sad
            
target_info

{'기사명': '회사 쪼개는 SK텔레콤, 통신 넘어 비통신 빅테크로 거듭날까',
 '날짜': '2021. 04. 24. 20:03',
 '댓글': ['ICT 투자전문회사\n최종적으로 SK지주랑 인수합병하는거겠지요?'],
 '좋아요': '0',
 '화나요': '1',
 '슬퍼요': '0'}

## 전체 기사 for문으로 데이터 수집하기

In [22]:
len(df['url'])

24

In [23]:
dict = {}

# 페이지당 기사 수집
for i in tqdm(range(0, 24)):   # len(df['url'])
    try:
        # 뉴스 크롬창 띄우기
        driver = webdriver.Chrome("chromedriver.exe")  # 맥 : /Users/aiden/Desktop/chromedriver
        driver.get(df['url'][i])
        time.sleep(1)

        # 기사 데이터 수집
        title = driver.find_element_by_css_selector('.tit_view').text
        date = driver.find_element_by_css_selector('.num_date').text
        angry = driver.find_element_by_css_selector(".emotion_list .selectionbox.type-LIKE.unselected .count").text
        like = driver.find_element_by_css_selector(".emotion_list .selectionbox.type-ANGRY.unselected .count").text
        sad = driver.find_element_by_css_selector(".emotion_list .selectionbox.type-SAD.unselected .count").text
        
        # 기사 댓글 갯수 
        review_count = int(driver.find_element_by_css_selector(".num_count").text.replace(',', ''))

        # review 수집하기
        review_list = []
        overlays1 = ".desc_txt"
        reviews = driver.find_elements_by_css_selector(overlays1)
        for review in tqdm(reviews):    
            review = review.text
            review_list.append(review)

        target_info = {}
        target_info['기사명'] = title
        target_info['날짜'] = date
        target_info['좋아요'] = like
        target_info['화나요'] = angry
        target_info['슬퍼요'] = sad
        target_info['댓글 수'] = len(review_list)
        target_info['댓글'] = review_list

        dict[i] = target_info
        
        print(title, '( 댓글 수:', len(review_list),')')

        driver.close()
        time.sleep(1)
        
    except:
        driver.close()
        continue

print(len(dict))
dict


SK텔레콤 '의료 AI' 개발 협약 ( 댓글 수: 0 )



SK텔레콤-가톨릭중앙의료원, AI 기반 의료 서비스 제공 ( 댓글 수: 1 )



SK텔레콤, 가톨릭중앙의료원과 의료 AI 솔루션 개발 협약 체결 ( 댓글 수: 1 )



SK텔레콤 'K팝 메타버스 프로젝트' 추진 ( 댓글 수: 1 )



SK텔레콤, 혼합현실 기술 활용 'K팝 메타버스 프로젝트' 시행 ( 댓글 수: 1 )


...


SK텔레콤 '퀀텀 키퍼' 시즌2 진행 ( 댓글 수: 0 )



SK텔레콤, 양자 보안 기술 쉽게 알리는 '퀀텀 키퍼' 시즌2 진행 ( 댓글 수: 1 )



회사 쪼개는 SK텔레콤, 통신 넘어 비통신 빅테크로 거듭날까 ( 댓글 수: 1 )



'탈(脫)네이버' 전략 검토하는 SK텔레콤..이베이 인수전 3社3色 ( 댓글 수: 1 )

24


{0: {'기사명': "SK텔레콤 '의료 AI' 개발 협약",
  '날짜': '2021. 04. 30. 14:40',
  '좋아요': '0',
  '화나요': '0',
  '슬퍼요': '0',
  '댓글 수': 0,
  '댓글': []},
 1: {'기사명': 'SK텔레콤-가톨릭중앙의료원, AI 기반 의료 서비스 제공',
  '날짜': '2021. 04. 30. 09:35',
  '좋아요': '0',
  '화나요': '0',
  '슬퍼요': '1',
  '댓글 수': 1,
  '댓글': ['ㅇ']},
 2: {'기사명': 'SK텔레콤, 가톨릭중앙의료원과 의료 AI 솔루션 개발 협약 체결',
  '날짜': '2021. 04. 30. 08:26',
  '좋아요': '0',
  '화나요': '1',
  '슬퍼요': '0',
  '댓글 수': 1,
  '댓글': ['dd']},
 3: {'기사명': "SK텔레콤 'K팝 메타버스 프로젝트' 추진",
  '날짜': '2021. 04. 29. 10:19',
  '좋아요': '0',
  '화나요': '1',
  '슬퍼요': '0',
  '댓글 수': 1,
  '댓글': ['ㅇㅇ']},
 4: {'기사명': "SK텔레콤, 혼합현실 기술 활용 'K팝 메타버스 프로젝트' 시행",
  '날짜': '2021. 04. 29. 14:43',
  '좋아요': '0',
  '화나요': '1',
  '슬퍼요': '0',
  '댓글 수': 1,
  '댓글': ['ddddd']},
 5: {'기사명': "SK텔레콤, 'K팝 메타버스 프로젝트' 시행",
  '날짜': '2021. 04. 29. 09:11',
  '좋아요': '1',
  '화나요': '0',
  '슬퍼요': '0',
  '댓글 수': 1,
  '댓글': ['dddddd']},
 6: {'기사명': "둘로 쪼갠 SK텔레콤..새 이름 'SKT-SKT스퀘어?' [IT선빵!]",
  '날짜': '2021. 04. 29. 13:54',
  '좋아요': '0',
  '화나요': '1',
  '

In [24]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df
# result_df.rename(columns={result_df.columns[0] : name}, inplace=True)

,기사명,날짜,좋아요,화나요,슬퍼요,댓글 수,댓글
0,SK텔레콤 '의료 AI' 개발 협약,2021. 04. 30. 14:40,0,0,0,0,[]
1,"SK텔레콤-가톨릭중앙의료원, AI 기반 의료 서비스 제공",2021. 04. 30. 09:35,0,0,1,1,[ㅇ]
2,"SK텔레콤, 가톨릭중앙의료원과 의료 AI 솔루션 개발 협약 체결",2021. 04. 30. 08:26,0,1,0,1,[dd]
3,SK텔레콤 'K팝 메타버스 프로젝트' 추진,2021. 04. 29. 10:19,0,1,0,1,[ㅇㅇ]
4,"SK텔레콤, 혼합현실 기술 활용 'K팝 메타버스 프로젝트' 시행",2021. 04. 29. 14:43,0,1,0,1,[ddddd]
5,"SK텔레콤, 'K팝 메타버스 프로젝트' 시행",2021. 04. 29. 09:11,1,0,0,1,[dddddd]
6,둘로 쪼갠 SK텔레콤..새 이름 'SKT-SKT스퀘어?' [IT선빵!],2021. 04. 29. 13:54,0,1,0,1,[ㅇㅇ]
7,"SK텔레콤, MS·SAP와 스타트업 지원 'ESG 얼라이언스' 결성",2021. 04. 29. 14:02,1,0,0,1,[ㅇㅇㅇ]
8,"SK텔레콤, ESG 코리아 2021 얼라이언스 결성",2021. 04. 29. 08:20,0,0,1,1,[asdasd]
9,"SK텔레콤, ESG 관련 스타트업 성장 지원하는 글로벌 얼라이언스 결성",2021. 04. 29. 08:20,0,1,0,0,[]


In [25]:
# 파일로 저장
result_df.to_excel("daum_article_({}).xlsx".format(keyword))